<a href="https://colab.research.google.com/github/llucas-Santos/santander-dev-week-2023-api/blob/main/SantanderDevWeek2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [9]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [7]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[1340, 1341, 1342, 1004]


In [10]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 1340,
    "name": "leticia",
    "account": {
      "id": 1402,
      "number": "60672857-9",
      "agency": "1544",
      "balance": 5000.0,
      "limit": 10000.0
    },
    "card": {
      "id": 1280,
      "number": "5586 1015 0956 9767",
      "limit": 10000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 1341,
    "name": "fernanda",
    "account": {
      "id": 1403,
      "number": "46481890-9",
      "agency": "4261",
      "balance": 5000.0,
      "limit": 10000.0
    },
    "card": {
      "id": 1281,
      "number": "4716 7492 0897 4716",
      "limit": 10000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 1342,
    "name": "debora",
    "account": {
      "id": 1404,
      "number": "0409969-9",
      "agency": "2494",
      "balance": 5000.0,
      "limit": 10000.0
    },
    "card": {
      "id": 1282,
      "number": "5417 8847 9690 0408",
      "limit": 10000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 1

## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [11]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.4 MB/s eta 0:00:00


In [12]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
openai_api_key = 'TODO'

In [18]:
import random

with open('msgRandom.txt', 'r') as arquivo:
  mensagens = arquivo.readlines()


def generate_ai_news(user):
  mensagem = random.choice(mensagens).strip()
  text = (f"Ola {user['name']}, {mensagem}")
  return text


for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({"description": news})

Ola leticia, Coloque seu dinheiro para crescer.
Ola fernanda, Investimento é liberdade.
Ola debora, O investimento é a chave para a prosperidade.
Ola roberta, Invista no seu potencial.


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [19]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User leticia updated? True!
User fernanda updated? True!
User debora updated? True!
User roberta updated? True!
